In [1]:
from datalake_tools import connection_data_analytics # biblioteca para conexão no Data Lake
import pandas as pd # biblioteca para manipulação de dados tabulados
import numpy as np # biblioteca com funções matemáticas, lógicas, etc
from datetime import date, datetime # biblioteca com funções de data
from dateutil.relativedelta import relativedelta # função para cálculo de diferenças de datas
import io # Funções de interface input/output com o sistema opercional
from os import getlogin # Função para retornar o usuário logado
import pathlib # Funções para manipulação de arquivos e diretórios
import reportlab
from fpdf import FPDF
from PyPDF2 import PdfFileWriter, PdfFileReader
import matplotlib as mpl
import matplotlib.pyplot as plt # Classe base para criação de gráficos da biblioteca matplotlib
import matplotlib.ticker as ticker
from matplotlib.ticker import StrMethodFormatter
import locale
import re
from svglib.svglib import svg2rlg # Classe para manipulação de arquivos svg (Scalable Vector Graphics)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
pd.set_option("display.max_columns", None)

In [ ]:
def truncate(num):
    if num >= 0:
        trunc = re.sub(r'^(\d+\.\d{,2})\d*$',r'\1',str(num))
    else:
        num = -1*num
        trunc = re.sub(r'^(\d+\.\d{,2})\d*$',r'\1',str(num))
        trunc = '-'+trunc
    tam_antes_ponto = trunc.find('.')
    tam_total = len(trunc)
    tam_depois_ponto = tam_total - tam_antes_ponto - 1  
    
    if tam_depois_ponto < 2:
        trunc = trunc+'0' 
    return trunc


este_mes = (datetime.today().strftime("%B")).upper()
este_mes_num = (datetime.today().strftime("%m"))
este_ano = (datetime.today().strftime("%Y")).upper()

cod_distribuidora = input("Informe o Código da Distribuidora:")

tipo = input("Qual o tipo de evento? (1)RTA (2)RTP (3)RTE")

data_publi = input("Informe a data de publicação no formato dd/mm/aaaa")

ano_publi = data_publi[6:]
efeito_medio = input("Informe o efeito médio no formato xx,xx%")


#Filtrando as linhas da distribuidora selecionada
df_impacto1 = df_impacto.loc[df_impacto['cd_distribuidor'] == cod_distribuidora]
#criando uma lista de códigos loja dessa distribuidora para iterar na geração de relatórios
sr_unidades_ait = df_impacto1['cod_loja']
unidades_total = sr_unidades_ait.size

dist = df_impacto1['distribuidora'].iloc[0]
res = df_impacto1['num_resolucao'].iloc[0]
res = res+'/'+ano_publi
ini = df_impacto1['dt_inicio_vigencia'].iloc[0].strftime("%x")
fim = df_impacto1['dt_fim_vigencia'].iloc[0].strftime("%x")

#Criando a pasta da distribuidora

path_destino = f"C:\\Users\\{getlogin()}\\Documents\\{dist}\\"
pathlib.Path(path_destino).mkdir(parents=True, exist_ok=True)    

pathlib.Path().mkdir(parents=True, exist_ok=True) 

if tipo == "1":
    tipo = "RTA"
    tipo_ext = "Reajuste Tarifário Anual"
    prep = 'do'
elif tipo == '2':
    tipo = "RTP"
    tipo_ext = "Revisão Tarifária Peródica"
    prep = 'da'
elif tipo == '3':
    tipo = "RTE"
    tipo_ext = "Revisão Tarifária Extraordinária"
    prep = 'da'
else:
    tipo = ""
    tipo_ext = ""



#Criando o background comum do relatório
#definindo largura e altura da página
wdt=210
hgt=297
nl = "\n"

#Criando o texto explicativo

texto = (f'Em {data_publi}, foi publicada a Resolução Homologatória nº {res}, editada pela Agência Nacional de Energia Elétrica (ANEEL), que homologa o resultado {prep} {tipo_ext} - {tipo} de {ano_publi} da {dist}, com vigência para o período de {ini} a {fim}.{nl}O efeito médio a ser percebido, considerando todos os consumidores em alta e baixa tensão, é de {efeito_medio}.')

pdf = FPDF('P', 'mm', (wdt, hgt))
pdf.set_auto_page_break(auto=1, margin = 0.0)
pdf.set_right_margin(0.0)
pdf.add_page()

pdf.image('./resources/BGA44.jpg',0,0,wdt+1)



pdf.set_font('Source_Sans_Pro', '', 12)
pdf.set_text_color(255,255,255)

#Adicionando o título
pdf.set_xy(52, 4.3)
pdf.cell(w =70, h = 7.7, txt = 'ANÁLISE DE IMPACTO TARIFÁRIO', border = 0, ln = 0, align = 'L', fill = False)

#adiciona o mes
pdf.set_font('Source_Sans_Pro', '', 11)

pdf.set_xy(160,5.5)
pdf.cell(w = 44, h = 5, txt = f'{este_mes} | {este_ano}', border = 0, ln = 0, align = 'C', fill = False)


#Textos de com e Sem ICMS (fundo escuro)
pdf.set_font('Source_Sans_Pro', '', 12)
pdf.set_xy(19, 73)
pdf.write(7,'SEM ICMS')
pdf.set_xy(19.6, 80)
pdf.write(7,'COM ICMS')

#adicionando subtítulos
pdf.set_font('Source_Sans_Pro', '', 14)
pdf.set_text_color(0,0,0)

pdf.set_xy(15.5, 61.5)
pdf.write(7,'1')
pdf.set_xy(22, 61.5)
pdf.write(7,'Impacto na fatura')
         
pdf.set_xy(106.5, 61.5)
pdf.write(7,'2')
pdf.set_xy(113, 61.5)
pdf.write(7,'Custos de distribuição')

pdf.set_xy(15.5, 95.5)
pdf.write(5,'3')
pdf.set_xy(22, 95.5)
pdf.write(5,'Análise Gráfica do Impacto')

pdf.set_xy(126.5, 95.5)
pdf.write(5,'4')
pdf.set_xy(132, 95.5)
pdf.write(5,'Dados da unidade')

pdf.set_xy(15.5, 189.5)
pdf.write(5,'5')
pdf.set_xy(22, 189.5)
pdf.write(5,'Evolução de tarifas')

pdf.set_xy(15.5, 240)
pdf.write(5,'6')
pdf.set_xy(22, 240)
pdf.write(5,'Detalhes do cálculo')

#Textos de com e Sem ICMS (fundo claro)
pdf.set_font('Source_Sans_Pro', '', 12)
pdf.set_xy(110, 73)
pdf.write(7,'SEM ICMS')
pdf.set_xy(110.6, 80)
pdf.write(7,'COM ICMS')


pdf.set_font('Source_Sans_Pro_Light', '', 9)

pdf.set_xy(130,67)
pdf.cell(w = 31, h = 9.2, txt = 'ANTERIOR', border = 0, ln = 0, align = 'C', fill = False)

pdf.set_xy(161,67)
pdf.cell(w = 31, h = 9.2, txt = 'ATUAL', border = 0, ln = 0, align = 'C', fill = False)

pdf.set_font('Source_Sans_Pro_Light', '', 9)

pdf.set_xy(78,188.5)
pdf.cell(w = 43, h = 9.2, txt = 'ANTERIOR', border = 0, ln = 0, align = 'C', fill = False)

pdf.set_xy(121,188.5)
pdf.cell(w = 43, h = 9.2, txt = 'ATUAL', border = 0, ln = 0, align = 'C', fill = False)

#adiciona detalhes da resolução
pdf.set_font('Source_Sans_Pro', '', 11)
pdf.set_xy(14,29)
pdf.multi_cell(w = 184, h = 5, txt = texto, border = 0, align = 'L', fill = False)

pdf.set_font('Source_Sans_Pro_Bold', '', 11)
pdf.set_xy(14,49)
pdf.multi_cell(w = 184, h = 5, txt = 'Confira abaixo a análise exclusiva aplicada ao seu negócio.', border = 0, align = 'L', fill = False)



# pdf.set_font('Source_Sans_Pro', '', 9)

#Cabeçalhos da tabela

pdf.set_font('Source_Sans_Pro_Light', '', 9)

pdf.set_xy(78,188.5)
pdf.cell(w = 43, h = 9.2, txt = 'ANTERIOR', border = 0, ln = 0, align = 'C', fill = False)

pdf.set_xy(121,188.5)
pdf.cell(w = 43, h = 9.2, txt = 'ATUAL', border = 0, ln = 0, align = 'C', fill = False)

pdf.set_xy(163,188.5)
pdf.cell(w = 32, h = 9.2, txt = 'VARIAÇÃO', border = 0, ln = 0, align = 'C', fill = False)



#Rótulos cadastrais

pdf.set_font('Source_Sans_Pro_Light', '', 9)

pdf.set_xy(125,106)
rotulo = f'CLASSE E{nl}MODALIDADE'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)

pdf.set_xy(160,106)
rotulo = f'DESCONTO{nl}APLICADO'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)

pdf.set_xy(125,132)
rotulo = f'DEMANDA CONTRATADA{nl}PONTA (kW)'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)

pdf.set_xy(160,132)
rotulo = f'DEMANDA CONTRATADA{nl}FORA PONTA (kW)'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)

pdf.set_xy(125,158)
rotulo = f'CONSUMO{nl}PONTA (MWh)'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)

pdf.set_xy(160,158)
rotulo = f'CONSUMO{nl}FORA PONTA (MWh)'
pdf.multi_cell(w = 36, h = 4, txt = rotulo, border = 0, align = 'C', fill = False)


#Rodapé
pdf.set_xy(89, 278)
pdf.set_font('Source_Sans_Pro_Bold', '', 7)
pdf.write(5, 'Canal de Ética')

pdf.set_xy(89, 282)
pdf.set_font('Source_Sans_Pro', '', 7)


pdf.set_xy(24, 278)
pdf.set_font('Source_Sans_Pro_Bold', '', 7)
pdf.write(5, 'Experiência do Cliente')
pdf.set_font('Source_Sans_Pro', '', 7)
pdf.set_xy(24, 282)



#salva a página

pdf.output('template.pdf', 'F')

contador = 0

#adicionando as informações de cada unidade e gerando o relatório
for cod_loja in sr_unidades_ait:
    pdf1 = FPDF('P', 'mm', (wdt, hgt))
    pdf1.add_page()
    pdf1.set_auto_page_break(auto=1, margin = 0.0)

    
    #transformando a df com uma linha em uma serie
    uni = df_impacto1
    uni = uni.loc[df_impacto1['cod_loja'] == cod_loja].squeeze()
    #transformando a serie em dicionario
    #esses dois passos servem para não incluir o índice no dicionário
    uni = uni.to_dict()
    
    unidade = uni['unidade']
    desconto = str(round(uni['desconto_resultante']*100,3)).replace('.',',')
    modalidade = uni['modalidade_tarifaria']
    cp = round(uni['consumo_ponta'],3)
    cp = (f'{cp:,}').replace(',','@').replace('.',',').replace('@','.')
    cfp = round(uni['consumo_fora_ponta'],3)
    cfp = (f'{cfp:,}').replace(',','@').replace('.',',').replace('@','.')
    dp = np.floor(uni['demanda_contratada_ponta'])
    if modalidade == 'azul':
        dp = (f'{dp:,}').replace(',','@').replace('.',',').replace('@','.')
        dp = dp[:-2]
    else:
        dp = '-'
    dfp = np.floor(uni['demanda_contratada_fora_ponta'])
    dfp = (f'{dfp:,}').replace(',','@').replace('.',',').replace('@','.')
    dfp = dfp[:-2]
    clas_mod = uni['cd_classe_tensao']+" "+uni['modalidade_tarifaria']
    
    contador = contador + 1
    print(f'{contador}/{unidades_total} - {unidade}')
    
    
##     valores atuais
    impostos = uni['vl_tributos_dist']
    encargos_ponta = uni['vl_tusd_mwh_ponta'] + uni['vl_cde_covid_ponta']
    encargos_fora_ponta = uni['vl_tusd_mwh_fora_ponta'] + uni['vl_cde_covid_fora_ponta']
    demanda_ponta = uni['vl_tusd_kw_ponta']
    demanda_fora_ponta = uni['vl_tusd_kw_fora_ponta']
    encargos = encargos_ponta + encargos_fora_ponta
    sem_icms = float(truncate(uni['vl_dist_sem_icms']))
    sem_icms = (f'{sem_icms:,}').replace(',','@').replace('.',',').replace('@','.')
    com_icms = float(truncate(uni['vl_dist_com_tributos']))
    com_icms = (f'{com_icms:,}').replace(',','@').replace('.',',').replace('@','.')
    
# #    valores anteriores
    impostos_ant = uni['vl_tributos_dist_ant']
    encargos_ponta_ant = uni['vl_tusd_mwh_ponta_ant'] + uni['vl_cde_covid_ponta_ant']
    encargos_fora_ponta_ant = uni['vl_tusd_mwh_fora_ponta_ant'] + uni['vl_cde_covid_fora_ponta_ant']
    demanda_ponta_ant = uni['vl_tusd_kw_ponta_ant']
    demanda_fora_ponta_ant = uni['vl_tusd_kw_fora_ponta_ant']
    encargos_ant = encargos_ponta_ant + encargos_fora_ponta_ant    
    sem_icms_ant = float(truncate(uni['vl_dist_sem_icms_ant']))
    sem_icms_ant = (f'{sem_icms_ant:,}').replace(',','@').replace('.',',').replace('@','.')
    com_icms_ant = float(truncate(uni['vl_dist_com_tributos_ant']))
    com_icms_ant = (f'{com_icms_ant:,}').replace(',','@').replace('.',',').replace('@','.')
    
#     impacto na fatura
    impacto_sem_icms = float(truncate(uni['delta_dist_sem_icms']))
    impacto_sem_icms = (f'{impacto_sem_icms:,}').replace(',','@').replace('.',',').replace('@','.')
    
    impacto_com_icms = float(truncate(uni['delta_dist_com_tributos']))
    impacto_com_icms = (f'{impacto_com_icms:,}').replace(',','@').replace('.',',').replace('@','.')
    
    percent_sem_icms = truncate(uni['delta%_dist_sem_icms']*100).replace('.',',')
    
    percent_com_icms = truncate(uni['delta%_dist_com_tributos']*100).replace('.',',')
    
#     impacto nas tarifas
    tarifa_kw_ponta_ant = truncate(uni['tusd_kw_ponta_ant']).replace('.',',')
    tarifa_kw_ponta = truncate(uni['tusd_kw_ponta']).replace('.',',')
    percent_tusd_kw_ponta = truncate(uni['delta%_tusd_kw_ponta_tarifa']*100).replace('.',',')+'%'
    
    tarifa_kw_fora_ponta_ant = truncate(uni['tusd_kw_fora_ponta_ant']).replace('.',',')
    tarifa_kw_fora_ponta = truncate(uni['tusd_kw_fora_ponta']).replace('.',',')
    percent_tusd_kw_fora_ponta = truncate(uni['delta%_tusd_kw_fora_ponta_tarifa']*100).replace('.',',')+'%'

    tarifa_mwh_ponta_ant = truncate(uni['tusd_mwh_ponta_ant']).replace('.',',')
    tarifa_mwh_ponta = truncate(uni['tusd_mwh_ponta']).replace('.',',')
    percent_tusd_mwh_ponta = truncate(uni['delta%_tusd_mwh_ponta_tarifa']*100).replace('.',',')+'%'
    
    tarifa_mwh_fora_ponta_ant = truncate(uni['tusd_mwh_fora_ponta_ant']).replace('.',',')
    tarifa_mwh_fora_ponta = truncate(uni['tusd_mwh_fora_ponta']).replace('.',',')
    percent_tusd_mwh_fora_ponta = truncate(uni['delta%_tusd_mwh_fora_ponta_tarifa']*100).replace('.',',')+'%'
    
    tarifa_cde_covid_ant = truncate(uni['cde_covid_mwh_ant']).replace('.',',')
    tarifa_cde_covid = truncate(uni['cde_covid_mwh']).replace('.',',')
    percent_cde_covid = truncate(uni['delta%_cde_covid_tarifa']*100).replace('.',',')+'%'
    
#     Adiciona o nome da unidade
    pdf1.set_font('Source_Sans_Pro', '', 12)
    pdf1.set_text_color(255,255,255)
    pdf1.set_xy(52,12)
    pdf1.write(5, unidade)

    
#     Preenchendo campos superiores

    pdf1.set_font('Source_Sans_Pro', '', 12)
    pdf1.set_text_color(255,255,255)
    pdf1.set_xy(46, 73.5)
    pdf1.cell(w = 27, h = 6, txt = 'R$ '+impacto_sem_icms, border = 0, ln = 0, align = 'C', fill = False)
    
    pdf1.set_xy(76,73.5)
    pdf1.cell(w = 22, h = 6, txt = f'{percent_sem_icms}%', border = 0, ln = 0, align = 'C', fill = False)
    
    pdf1.set_xy(46,80.5)
    pdf1.cell(w = 27, h = 6, txt = 'R$ '+impacto_com_icms, border = 0, ln = 0, align = 'C', fill = False)

    pdf1.set_xy(76,80.5)
    pdf1.cell(w = 22, h = 6, txt = f'{percent_com_icms}%', border = 0, ln = 0, align = 'C', fill = False)
    
    pdf1.set_text_color(0,0,0)
    pdf1.set_xy(130, 73.5)
    pdf1.cell(w = 31, h = 6, txt = 'R$ '+sem_icms_ant, border = 0, ln = 0, align = 'C', fill = False)
    
    pdf1.set_xy(161,73.5)
    pdf1.cell(w = 31, h = 6, txt = 'R$ '+sem_icms, border = 0, ln = 0, align = 'C', fill = False)
    
    pdf1.set_xy(130,80.5)
    pdf1.cell(w = 31, h = 6, txt = 'R$ '+com_icms_ant, border = 0, ln = 0, align = 'C', fill = False)

    pdf1.set_xy(161,80.5)
    pdf1.cell(w = 31, h = 6, txt = 'R$ '+com_icms, border = 0, ln = 0, align = 'C', fill = False)  

    
    
#   Preenchendo os dados cadastrais
    pdf1.set_font('Source_Sans_Pro_Bold', '', 10.5)

    pdf1.set_xy(125,118)
    dado = clas_mod
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)

    pdf1.set_xy(160,118)
    dado = desconto+' %'
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)

    pdf1.set_xy(125,144)
    dado = dp
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)

    pdf1.set_xy(160,144)
    dado = dfp
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)

    pdf1.set_xy(125,170)
    dado = cp
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)

    pdf1.set_xy(160,170)
    dado = cfp
    pdf1.multi_cell(w = 36, h = 5, txt = dado, border = 0, align = 'C', fill = False)



# Tabela de tarifas

    pdf1.set_font('Source_Sans_Pro', '', 10.5)
    pdf1.set_text_color(0,0,0)
    
    if modalidade == 'azul':
        pdf1.set_xy(16,196)
        pdf1.cell(w= 62, h = 9, txt = 'DEMANDA PONTA (R$/kW)', align = 'C')
        pdf1.set_xy(78,196)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_ponta_ant, align = 'C')
        pdf1.set_xy(121,196)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_ponta, align = 'C')
        pdf1.set_xy(164,196)
        if uni['delta%_tusd_kw_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_kw_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,204.5)
        pdf1.cell(w= 62, h = 9, txt = 'DEMANDA FORA PONTA (R$/kW)', align = 'C')
        pdf1.set_xy(78,204.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_fora_ponta_ant, align = 'C')
        pdf1.set_xy(121,204.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_fora_ponta, align = 'C')
        pdf1.set_xy(164,204.5)
        if uni['delta%_tusd_kw_fora_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_kw_fora_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,213)
        pdf1.cell(w= 62, h = 9, txt = 'ENCARGOS (R$/MWh)', align = 'C')
        pdf1.set_xy(78,213)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_fora_ponta_ant, align = 'C')
        pdf1.set_xy(121,213)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_fora_ponta, align = 'C')
        pdf1.set_xy(164,213)
        if uni['delta%_tusd_mwh_fora_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_mwh_fora_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,221.5)
        pdf1.cell(w= 62, h = 9, txt = 'CDE COVID (R$/MWh)', align = 'C')
        pdf1.set_xy(78,221.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_cde_covid_ant, align = 'C')
        pdf1.set_xy(121,221.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_cde_covid, align = 'C')
        pdf1.set_xy(164,221.5)
        if uni['delta%_cde_covid_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_cde_covid, align = 'C')
        pdf1.set_text_color(0,0,0)
    else:
        pdf1.set_xy(16,196)
        pdf1.cell(w= 62, h = 9, txt = 'DEMANDA (R$/kW)', align = 'C')
        pdf1.set_xy(78,196)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_fora_ponta_ant, align = 'C')
        pdf1.set_xy(121,196)
        pdf1.cell(w= 43, h = 9, txt = tarifa_kw_fora_ponta, align = 'C')
        pdf1.set_xy(164,196)
        if uni['delta%_tusd_kw_fora_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_kw_fora_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,204.5)
        pdf1.cell(w= 62, h = 9, txt = 'ENCARGOS PONTA (R$/MWh)', align = 'C')
        pdf1.set_xy(78,204.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_ponta_ant, align = 'C')
        pdf1.set_xy(121,204.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_ponta, align = 'C')
        pdf1.set_xy(164,204.5)
        if uni['delta%_tusd_mwh_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_mwh_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,213)
        pdf1.cell(w= 62, h = 9, txt = 'ENCARGOS FORA PONTA (R$/MWh)', align = 'C')
        pdf1.set_xy(78,213)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_fora_ponta_ant, align = 'C')
        pdf1.set_xy(121,213)
        pdf1.cell(w= 43, h = 9, txt = tarifa_mwh_fora_ponta, align = 'C')
        pdf1.set_xy(164,213)
        if uni['delta%_tusd_mwh_fora_ponta_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_tusd_mwh_fora_ponta, align = 'C')
        pdf1.set_text_color(0,0,0)
        pdf1.set_xy(16,221.5)
        pdf1.cell(w= 62, h = 9, txt = 'CDE COVID (R$/MWh)', align = 'C')
        pdf1.set_xy(78,221.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_cde_covid_ant, align = 'C')
        pdf1.set_xy(121,221.5)
        pdf1.cell(w= 43, h = 9, txt = tarifa_cde_covid, align = 'C')
        pdf1.set_xy(164,221.5)
        if uni['delta%_cde_covid_tarifa'] > 0:
            pdf1.set_text_color(172,0,0)
        else:
            pdf1.set_text_color(4,171,11)
        pdf1.cell(w= 32, h = 9, txt = percent_cde_covid, align = 'C')
        pdf1.set_text_color(0,0,0)

    
#     Adicionando o testo da legenda dos gráficos
    
    pdf1.set_font('Source_Sans_Pro_Light', '', 7)
    pdf1.set_xy(105,104.5)
    pdf1.cell(w= 10, h = 9, txt = 'Anterior', align = 'L')
    pdf1.set_xy(105,109)
    pdf1.cell(w= 10, h = 9, txt = 'Atual', align = 'L')
    
#     Gerando o grafico
    plt.rcParams['axes.formatter.use_locale'] = True
    
    
    # Cria os objetos figure e axis (obs.: frameon=False --> gráfico sem frame em volta)
    
    fig, ax = plt.subplots(figsize=(15, 12), frameon=False)
    plt.tight_layout()
   
    pdf1.set_font('Source_Sans_Pro_Light', '', 9)
    
    if modalidade == 'verde':
        itens = ['IMPOSTOS', 'ENCARGOS FP', 'ENCARGOS P', 'DEMANDA']
        custos = [impostos, encargos_fora_ponta, encargos_ponta, demanda_fora_ponta]
        custos_ant = [impostos_ant, encargos_fora_ponta_ant, encargos_ponta_ant, demanda_fora_ponta_ant]
        pdf1.set_xy(15.8,103)
        pdf1.cell(w= 43, h = 9, txt = 'Demanda¹', align = 'L')
        pdf1.set_xy(15.8,121.5)
        pdf1.cell(w= 43, h = 9, txt = 'Encargos Ponta²', align = 'L')
        pdf1.set_xy(15.8,139.5)
        pdf1.cell(w= 43, h = 9, txt = 'Encargos Fora Ponta²', align = 'L')
        pdf1.set_xy(15.8,158)
        pdf1.cell(w= 43, h = 9, txt = 'ICMS e PIS/COFINS³', align = 'L')
    else:
        itens = ['IMPOSTOS', 'ENCARGOS', 'DEMANDA P', 'DEMANDA FP']
        custos = [impostos, encargos, demanda_ponta, demanda_fora_ponta]
        custos_ant = [impostos_ant, encargos_ant, demanda_ponta_ant, demanda_fora_ponta_ant]
        pdf1.set_xy(15.8,103)
        pdf1.cell(w= 43, h = 9, txt = 'Demanda Fora Ponta¹', align = 'L')
        pdf1.set_xy(15.8,121.5)
        pdf1.cell(w= 43, h = 9, txt = 'Demanda Ponta¹', align = 'L')
        pdf1.set_xy(15.8,139)
        pdf1.cell(w= 43, h = 9, txt = 'Encargos²', align = 'L')
        pdf1.set_xy(15.8,157)
        pdf1.cell(w= 43, h = 9, txt = 'ICMS e PIS/COFINS³', align = 'L')
    X_axis = np.arange(len(itens))
    c1=ax.barh(X_axis, custos_ant, 0.3, label = 'Anterior', tick_label = itens, color = '#0c2632')
    
    
    
    
    ax.set_yticks([0.33, 1.33, 2.4, 3.43])
    ax.set_yticklabels('')
    
    ax.tick_params(bottom = False, left = False)

    plt.xticks([])
    
    c2=ax.barh(X_axis - 0.3, custos, 0.3, label = 'Atual', color = '#00df00')
    ax.set_frame_on(False)

    ax.bar_label(c1, fontsize=28, labels=[f' R$ {x:,.2f}'.translate(str.maketrans(',.', '.,')) for x in c1.datavalues])
    ax.bar_label(c2, fontsize=28, labels=[f' R$ {x:,.2f}'.translate(str.maketrans(',.', '.,')) for x in c2.datavalues])
#     plt.show()
    fig.savefig('grafico.png', bbox_inches='tight')
    
    plt.close()
    
    pdf1.image('grafico.png',16.4,106,104)
    
    
    
    
#     Salvando o pdf por cima do template
    pdf1.output('content.pdf', 'F')
    
    cod = uni['cod_loja'][:6]+uni['cod_loja'][-2:]
    nome_arquivo = f'{este_ano}{este_mes_num} - {cod} - {unidade} - {tipo}'
    caminho = f'{path_destino}/{nome_arquivo}'
    pdf_template = PdfFileReader(open('template.pdf', 'rb'))
    # Get the first page from the template
    template_page = pdf_template.getPage(0)
    # Open your overlay PDF that was created earlier
    overlay_pdf = PdfFileReader(open('content.pdf', 'rb'))
    # Merge the overlay page onto the template page
    template_page.mergePage(overlay_pdf.getPage(0))
    # Write the result to a new PDF file
    output_pdf = PdfFileWriter()
    output_pdf.addPage(template_page)
    output_pdf.write(open(f'{caminho}.pdf', "wb"))
    
pdf.close()
pdf1.close()

print('Processo finalizado')
